# 実行環境の作成

## python3.9

In [1]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

--2022-09-15 21:35:14--  https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  73.06M   247MB/s    in 0.3s    

2022-09-15 21:35:14 (247 MB/s) - ‘mini.sh’ saved [76607678/76607678]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - colorama==0.4.4=pyh

In [2]:
!python --version

Python 3.9.12


## Google Cloud Storage

下記コードでGCPに接続

In [3]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [4]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  70472      0 --:--:-- --:--:-- --:--:-- 70472
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,391 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://packages.cloud.google.com/apt gcsfuse-bionic/main amd64 Packages [1,443 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bion

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [5]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/15 21:39:03.827863 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/15 21:39:03.843056 Opening GCS connection...
2022/09/15 21:39:04.441649 Mounting file system "statistics-hyogo"...
2022/09/15 21:39:04.444043 File system has been successfully mounted.


## ESTAT_APP_ID

In [6]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [7]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

## 統計表のメタ情報を取得

'statsDataId'に統計表コードを指定

In [8]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [9]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
print(meta)

{'GET_META_INFO': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-16T06:39:14.544+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'DATA_FORMAT': 'J'}, 'METADATA_INF': {'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'COLLECT_AREA': '全国', 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 500976, 'UPDATED_DATE': '2022-03-04', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '都道府県データ', 'TABULATION_SUB_CATEGORY1': '基礎データ'}, 'DESCRIPTION': {'TABULATION_CATEGORY_EXPLANATION': '社会・人口統計体系の都道府県ごとに集計したデータを提供します。'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLASS_INF': {'CLASS_OBJ': [{'@id': 'tab', '@name': '観測値', 'CLASS': {'@code': '00001', 

## 統計表csvを作成

カテゴリコード'cat01'のリストを作成する関数

In [18]:
def generate_cards(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']
  print(STAT_NAME)

  TITLE = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['TITLE']['$']
  print(TITLE)

  # カテゴリ一覧
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  df = pd.json_normalize(cat01)[['@code','@name']]


  df['statsDataId'] = statsDataId
  df['statsDataName'] = f'{STAT_NAME}　「{TITLE}」'
  print(df)



  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

cards = generate_cards(statsDataId)
print(cards)

社会・人口統計体系
Ａ　人口・世帯
       @code                 @name statsDataId        statsDataName
0      A1101             A1101_総人口  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
1    A110101        A110101_総人口（男）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
2    A110102        A110102_総人口（女）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
3      A1102           A1102_日本人人口  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
4    A110201      A110201_日本人人口（男）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
..       ...                   ...         ...                  ...
563  A850205  A850205_父子世帯数（55歳以上）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
564    A9101            A9101_婚姻件数  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
565    A9111    A9111_平均婚姻年齢（初婚の夫）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
566    A9112    A9112_平均婚姻年齢（初婚の妻）  0000010101  社会・人口統計体系　「Ａ　人口・世帯」
567    A9201            A9201_離婚件数  0000010101  社会・人口統計体系　「Ａ　人口・世帯」

[568 rows x 4 columns]


NameError: ignored